In [1]:
import pandas as pd

In [3]:
from util import creating_main_df as func

# Reading and merging data from CVM
CVM is brazilians securities commission. 
On their website (http://dados.cvm.gov.br/dataset/fi-doc-eventual) you can find files that encapsulate all the reports sent by fund managers through the years. The files features are: fund type, employer ID number, fund name, document competence date, date of receipt of the document, link to download the file, file name and document type.

Files from 2015 to 04-26-2021 were appended.

In [9]:
df_cvm = func.merging_cvm('../data/portal_data/csv/')
df_cvm

,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,DT_RECEB,TP_DOC,NM_ARQ,ID_DOC,LINK_ARQ
0,FAPI,02.010.153/0001-45,BB FAPI FUNDO DE APOSENTADORIA PROGRAMADA INDI...,2019-12-31,2020-03-10,DEMONST CONTAB,20200310113210UP3f6ecb6624c745658f8a34ab64ac61...,NaN,http://sistemas.cvm.gov.br/docsrecebidos/20200...
1,FAPI,02.177.812/0001-32,ITAU FAPI RENDA FIXA - FUNDO DE APOSENTADORIA ...,2019-12-31,2020-03-02,DEMONST CONTAB,20200302164007UPcc7ca8cbfba1412d897304b1bf2c5e...,NaN,http://sistemas.cvm.gov.br/docsrecebidos/20200...
2,FAPI,02.177.815/0001-76,ITAU FAPI CONSERVADOR - FUNDO DE APOSENTADORIA...,2019-12-31,2020-03-02,DEMONST CONTAB,20200302164106UP951351fb8bfb45e7a7d55d575c4c78...,NaN,http://sistemas.cvm.gov.br/docsrecebidos/20200...
3,FAPI,02.185.027/0001-21,BRADESCO FUNDO DE APOSENTADORIA PROGRAMADA IND...,2019-12-31,2020-03-13,DEMONST CONTAB,20200313164431UP2bca9fcb3cfe4cefa5c82eaa61503f...,NaN,http://sistemas.cvm.gov.br/docsrecebidos/20200...
4,FAPI,02.226.122/0001-26,UNIBANCO FUNDO DE APOSENTADORIA PROGRAMADA IND...,2019-12-31,2020-02-28,DEMONST CONTAB,20200228185716UP88e6134308c3488884a0e3005b0926...,NaN,http://sistemas.cvm.gov.br/docsrecebidos/20200...
...,...,...,...,...,...,...,...,...,...
14138,FUNCINE,10.500.895/0001-87,INVESTIMAGE 1 FUNDO DE FINANCIAMENTO DA INDUST...,2021-03-10,2021-04-12,AGO,20210412142644UP94a750af962342ecbe5aac02ea8092...,NaN,http://sistemas.cvm.gov.br/docsrecebidos/20210...
14139,FUNCINE,10.500.895/0001-87,INVESTIMAGE 1 FUNDO DE FINANCIAMENTO DA INDUST...,2021-03-10,2021-04-09,REGUL FDO,20210409161247UP8c02dd755dff49edb564ee4130cf93...,NaN,http://sistemas.cvm.gov.br/docsrecebidos/20210...
14140,FUNCINE,10.500.895/0001-87,INVESTIMAGE 1 FUNDO DE FINANCIAMENTO DA INDUST...,2021-03-10,2021-04-12,REGUL FDO,20210412142644UP5ec32fba749a4a73a5718c4d5b4491...,NaN,http://sistemas.cvm.gov.br/docsrecebidos/20210...
14141,FUNCINE,11.179.262/0001-81,FUNCINE - BRB BRASÍLIA FUNCINE,2021-04-08,2021-04-08,AGO,20210408193318UP1dbf7ef9da244ae3b69a92ed5d7d82...,NaN,http://sistemas.cvm.gov.br/docsrecebidos/20210...


## Clean df_cvm

In [10]:
df_cvm=func.clean_cvm(df_cvm)

# Reading Economatica data
Economatica is an investment software. A table was created on their platform with the target values we want to predict.

In [12]:
df_eco= func.read_eco_data('../data/economatica21-05-24.xlsx')

# Rebalancing
There are many more funds that dont follow the resolution, rebalamcing was needed

Economatica has the resolution status on the present day, not from past reports. Research with specialists showed that funds don't usually change its status (98%).

To decrease the imbalance of our binomial distribution we decided to extend the status of the present report to  past documnets.



In [13]:
df_eco = func.rebalance_eco(df_eco)

# Merging & Fixing Target

with the premise that an Employer ID that follows Res 4661 has always followed it, the merge will happend only on CNPJ/EmployerID
used a dictionary com clean target to ones and zeros

In [16]:
df = func.merge_fix_target(df_cvm,df_eco)

## Analyses
Transforming complex files to plain will have total different routes depending on the type of file

In [14]:
doc=0
pdf=0
other=0
for i in list(cvm_and_eco['NM_ARQ']):
    if i[-3:].upper()=='DOC':
        doc+=1
    elif i[-3:].upper()=='PDF':
        pdf+=1
    else:
        print(i[-3:])
        
print('pdf',pdf)
print('doc',doc)
print('other',other)

pdf 16707
doc 2178
other 0


It is expected that files with the sabe Administrator will be simmilar

In [15]:
cvm_and_eco['Administrator'].value_counts()[:15]

Bem Dtvm                                    2496
BNY Mellon Servicos Financeiros Dtvm SA     2473
BTG Pactual Servicos Financeiros SA Dtvm    1419
Intrag                                      1379
Itau Unibanco SA                            1115
Singulare Corretora de Titulos e Valores     881
Santander Caceis Brasil Dtvm SA              796
Bradesco                                     726
-                                            662
Credit Suisse Hedging Griffo Cor Val S.A     659
Safra Servicos de Adm Fiduciaria             579
Banco Santander (Brasil) SA                  569
BB Dtvm S.A                                  529
BNP Paribas                                  505
Brl Dtvm                                     501
Name: Administrator, dtype: int64

# Create a df with one of each Administrator

In [18]:
administrator_df = cvm_and_eco.drop_duplicates(subset=['Administrator'])

In [19]:
administrator_df.to_json('../data/administrator_df.json')

## Now we have a Data Frame with all the info we needed to download the files and and find Target

In [17]:
df.to_json('../data/cvm_merge_eco.json')